In [2]:
import pandas as pd
!wget -q https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
df = pd.read_csv('titanic.csv')
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [4]:
import numpy as np
from sklearn.preprocessing import StandardScaler

X = df[['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']].copy()
y = df['Survived'].copy()

# Handle missing 'Age' values
X['Age'] = X['Age'].fillna(X['Age'].mean())

# Handle missing 'Embarked' values
mode_embarked = X['Embarked'].mode()[0]
X['Embarked'] = X['Embarked'].fillna(mode_embarked)

print("Missing values after handling:")
print(X.isnull().sum())

# One-hot encode 'Sex' and 'Embarked' columns
X = pd.get_dummies(X, columns=['Sex', 'Embarked'], drop_first=True)

# Scale numerical features ('Age' and 'Fare')
scaler = StandardScaler()
X[['Age', 'Fare']] = scaler.fit_transform(X[['Age', 'Fare']])

print("\nFeatures after encoding and scaling (first 5 rows):")
print(X.head())

Missing values after handling:
Pclass      0
Sex         0
Age         0
Fare        0
Embarked    0
dtype: int64

Features after encoding and scaling (first 5 rows):
   Pclass       Age      Fare  Sex_male  Embarked_Q  Embarked_S
0       3 -0.592481 -0.502445      True       False        True
1       1  0.638789  0.786845     False       False       False
2       3 -0.284663 -0.488854     False       False        True
3       1  0.407926  0.420730     False       False        True
4       3  0.407926 -0.486337      True       False        True


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the Logistic Regression model
model = LogisticRegression(solver='liblinear', random_state=42)

# Train the model
model.fit(X_train, y_train)

print("Logistic Regression model trained successfully.")

Logistic Regression model trained successfully.


In [6]:
from sklearn.metrics import classification_report

# Make predictions on the test set
y_pred = model.predict(X_test)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       105
           1       0.77      0.73      0.75        74

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



In [7]:
import joblib

# Save the trained model to disk
joblib.dump(model, 'logistic_regression_model.joblib')
print("Model saved as 'logistic_regression_model.joblib'")

# Load the saved model
loaded_model = joblib.load('logistic_regression_model.joblib')
print("Model loaded successfully.")

# Take a few samples from X_test
sample_data = X_test.head(5)
print("\nSample data from X_test:\n", sample_data)

# Make predictions using the loaded model on the sample data
loaded_predictions = loaded_model.predict(sample_data)
print("\nPredictions from loaded model on sample data:\n", loaded_predictions)

Model saved as 'logistic_regression_model.joblib'
Model loaded successfully.

Sample data from X_test:
      Pclass       Age      Fare  Sex_male  Embarked_Q  Embarked_S
709       3  0.000000 -0.341452      True       False       False
439       2  0.100109 -0.437007      True       False        True
840       3 -0.746389 -0.488854      True       False        True
720       2 -1.823750  0.016023     False       False        True
39        3 -1.208115 -0.422074     False       False       False

Predictions from loaded model on sample data:
 [0 0 0 1 1]
